In [26]:
import numpy as np
from gru import (
    # GRUModel,
    fibonacci_generator,
    one_by_n_generator,
    plus_one_minus_one_generator,
    squared_generator,
    half_generator,
    arithmetic_progression,
    create_sliding_window_batches,
    mape
)
from gru2 import GRUModel
import plotly.graph_objects as go

In [27]:
# seq_length = 10
# window_size = 5
# batch_size = 2
# hidden_size = 4
# output_size = 3

# sequence = list(fibonacci_generator(seq_length))
# X, y = create_sliding_window_batches(sequence, window_size, batch_size, output_size)
# X_train, y_train = X[0:int(len(X) * 0.8)], y[0:int(len(X) * 0.8)][-1]
# X_test, y_test = X[int(len(X) * 0.8):], y[int(len(X) * 0.8):][-1]

# verbosity = 1000

# max_mape = 1e-3
# learn_until_meet_mape = True

# model = GRUModel(window_size, hidden_size, output_size)

# # Параметры обучения
# lr = 1e-4
# epochs = 5000

# model.train(X_train, y_train, lr, epochs,
#             True, max_mape, verbosity)
# Проверим результат после обучения
# y_pred, _, _ = model.forward(X_train)
# print("Предсказание после обучения:\n", y_pred)
# print("Истинное значение:\n", y_train)
# print(f"MAPE: {mape(y_train, y_pred) * 100:.4f}%")

# print('TEST')
# y_pred, _, _ = model.forward(X_test)
# print("Предсказания на тестовой выборке:\n", y_pred)
# print("Истинное значение на тестовой выборке:\n", y_test)
# print(f"MAPE: {mape(y_test, y_pred) * 100:.4f}%")  

In [28]:
experiments_amount = 3
# epochs = np.linspace(1000, 35000, endpoint=True, num=10).astype(np.int32).tolist()
epochs = list(range(100, 2100, 100))

# np.random.seed(12345)

all_mapes = []
mapes_mean = []

all_epochs: list[int] = []

seq_length = 20
window_size = 5
batch_size = 2
hidden_size = 10
output_size = 3

verbosity = 100000
lr = 1e-4

sequence = list(arithmetic_progression(seq_length, a0=1, d=1))
X, y = create_sliding_window_batches(sequence, window_size, batch_size, output_size)

for max_epochs in epochs:    

    # Training network for five times
    mapes_sum = 0    
    for _ in range(experiments_amount):              
        
        network = GRUModel(input_size=window_size, hidden_size=hidden_size,
                           output_size=output_size)
        _, _mape = network.train(x=X, y=y, lr=lr, max_epochs=max_epochs, verbosity=verbosity)
        all_epochs.append(max_epochs)
        mapes_sum += _mape        
        all_mapes.append(_mape)    

    # Average mape for epochs amount
    mapes_mean.append(mapes_sum / experiments_amount)

TRAINING FINISHED
Epoch 100/100, Loss: 148.895041
MAPE: 0.871168
TRAINING FINISHED
Epoch 100/100, Loss: 142.125019
MAPE: 0.850335
TRAINING FINISHED
Epoch 100/100, Loss: 131.224154
MAPE: 0.810648
TRAINING FINISHED
Epoch 200/200, Loss: 54.595967
MAPE: 0.506671
TRAINING FINISHED
Epoch 200/200, Loss: 69.291897
MAPE: 0.566152
TRAINING FINISHED
Epoch 200/200, Loss: 77.818894
MAPE: 0.613255
TRAINING FINISHED
Epoch 300/300, Loss: 33.451721
MAPE: 0.378677
TRAINING FINISHED
Epoch 300/300, Loss: 21.021788
MAPE: 0.279702
TRAINING FINISHED
Epoch 300/300, Loss: 23.894597
MAPE: 0.305045
TRAINING FINISHED
Epoch 400/400, Loss: 8.048923
MAPE: 0.134468
TRAINING FINISHED
Epoch 400/400, Loss: 6.627333
MAPE: 0.127175
TRAINING FINISHED
Epoch 400/400, Loss: 11.334433
MAPE: 0.168753
TRAINING FINISHED
Epoch 500/500, Loss: 3.281322
MAPE: 0.100885
TRAINING FINISHED
Epoch 500/500, Loss: 3.483411
MAPE: 0.103565
TRAINING FINISHED
Epoch 500/500, Loss: 5.693667
MAPE: 0.117858
TRAINING FINISHED
Epoch 600/600, Loss: 2.5

In [29]:
# Plotting results for training epochs amount
fig = go.Figure()
fig.add_trace(
    go.Scatter(x=all_epochs, y=all_mapes,
               name='Все наблюдения', mode='markers')
)
fig.add_trace(
    go.Scatter(x=epochs, y=mapes_mean,
               name='Средние значения', mode='lines+markers')
)
fig.update_layout(
    title='Зависимость средней абсолютной ошибки от количества итераций обучения',
    font=dict(size=20),
    xaxis=dict(title='Количество итераций обучения'),
    yaxis=dict(title='Средняя абсолютная ошибка'),
    width=1280,
    height=720
)
fig.show()

In [30]:
experiments_amount = 3
# epochs = np.linspace(1000, 35000, endpoint=True, num=10).astype(np.int32).tolist()
hidden_sizes = list(range(2, 22, 2))

# np.random.seed(12345)

all_mapes = []
mapes_mean = []

all_sizes: list[int] = []

seq_length = 20
window_size = 5
batch_size = 2
output_size = 3

verbosity = 100000
max_mape = 1e-3
learn_until_meet_mape = True
lr = 1e-4
max_epochs = 250

sequence = list(arithmetic_progression(seq_length, a0=1, d=1))
X, y = create_sliding_window_batches(sequence, window_size, batch_size, output_size)
y = y

for size in hidden_sizes:    

    # Training network for five times
    mapes_sum = 0    
    for _ in range(experiments_amount):              
        
        network = GRUModel(input_size=window_size, hidden_size=size,
                           output_size=output_size)
        _, _mape = network.train(x=X, y=y, lr=lr, max_epochs=max_epochs, verbosity=verbosity)
        all_sizes.append(size)
        mapes_sum += _mape        
        all_mapes.append(_mape)    

    # Average mape for epochs amount
    mapes_mean.append(mapes_sum / experiments_amount)

TRAINING FINISHED
Epoch 250/250, Loss: 106.382437
MAPE: 0.709291
TRAINING FINISHED
Epoch 250/250, Loss: 95.534242
MAPE: 0.646926
TRAINING FINISHED
Epoch 250/250, Loss: 80.560949
MAPE: 0.610104
TRAINING FINISHED
Epoch 250/250, Loss: 119.020594
MAPE: 0.766286
TRAINING FINISHED
Epoch 250/250, Loss: 105.477072
MAPE: 0.717128
TRAINING FINISHED
Epoch 250/250, Loss: 101.270561
MAPE: 0.694258
TRAINING FINISHED
Epoch 250/250, Loss: 57.656446
MAPE: 0.514017
TRAINING FINISHED
Epoch 250/250, Loss: 47.441481
MAPE: 0.455294
TRAINING FINISHED
Epoch 250/250, Loss: 53.108265
MAPE: 0.484520
TRAINING FINISHED
Epoch 250/250, Loss: 33.103056
MAPE: 0.364990
TRAINING FINISHED
Epoch 250/250, Loss: 58.125573
MAPE: 0.512995
TRAINING FINISHED
Epoch 250/250, Loss: 37.486561
MAPE: 0.409033
TRAINING FINISHED
Epoch 250/250, Loss: 22.442345
MAPE: 0.303588
TRAINING FINISHED
Epoch 250/250, Loss: 25.160862
MAPE: 0.312959
TRAINING FINISHED
Epoch 250/250, Loss: 55.684188
MAPE: 0.497407
TRAINING FINISHED
Epoch 250/250, Los

In [31]:
# Plotting results for training epochs amount
fig = go.Figure()
fig.add_trace(
    go.Scatter(x=all_sizes, y=all_mapes,
               name='Все наблюдения', mode='markers')
)
fig.add_trace(
    go.Scatter(x=hidden_sizes, y=mapes_mean,
               name='Средние значения', mode='lines+markers')
)
fig.update_layout(
    title='Зависимость средней абсолютной ошибки от<br>количества нейронов на скрытом слое',
    font=dict(size=20),
    xaxis=dict(title='Количество нейронов на скрытом слое'),
    yaxis=dict(title='Средняя абсолютная ошибка'),
    width=1280,
    height=720
)
fig.show()

In [35]:
# MAPE relation with sliding window
experiments_amount = 3
# epochs = np.linspace(1000, 35000, endpoint=True, num=10).astype(np.int32).tolist()


all_mapes = []
mapes_mean = []

all_sizes: list[int] = []

seq_length = 50
window_sizes = list(range(1, seq_length // 4))
hidden_size = 10
batch_size = 2
output_size = 3

verbosity = 100000
max_mape = 1e-3
learn_until_meet_mape = True
lr = 1e-4
max_epochs = 1000

sequence = list(arithmetic_progression(seq_length, a0=1, d=1))

for size in window_sizes:
    X, y = create_sliding_window_batches(sequence, size, batch_size, output_size)    

    # Training network for five times
    mapes_sum = 0    
    for _ in range(experiments_amount):              
        
        network = GRUModel(input_size=size, hidden_size=hidden_size,
                           output_size=output_size)
        _, _mape = network.train(x=X, y=y, lr=lr, max_epochs=max_epochs, verbosity=verbosity)
        all_sizes.append(size)
        mapes_sum += _mape        
        all_mapes.append(_mape)    

    # Average mape for epochs amount
    mapes_mean.append(mapes_sum / experiments_amount)

TRAINING FINISHED
Epoch 1000/1000, Loss: 29.387974
MAPE: 0.287007
TRAINING FINISHED
Epoch 1000/1000, Loss: 29.469851
MAPE: 0.279693
TRAINING FINISHED
Epoch 1000/1000, Loss: 30.767441
MAPE: 0.280070
TRAINING FINISHED
Epoch 1000/1000, Loss: 21.877116
MAPE: 0.249659
TRAINING FINISHED
Epoch 1000/1000, Loss: 19.543882
MAPE: 0.242394
TRAINING FINISHED
Epoch 1000/1000, Loss: 27.083069
MAPE: 0.262604
TRAINING FINISHED
Epoch 1000/1000, Loss: 22.499265
MAPE: 0.230813
TRAINING FINISHED
Epoch 1000/1000, Loss: 24.643477
MAPE: 0.251527
TRAINING FINISHED
Epoch 1000/1000, Loss: 21.068834
MAPE: 0.225093
TRAINING FINISHED
Epoch 1000/1000, Loss: 7.646764
MAPE: 0.120330
TRAINING FINISHED
Epoch 1000/1000, Loss: 16.548425
MAPE: 0.181038
TRAINING FINISHED
Epoch 1000/1000, Loss: 14.014796
MAPE: 0.166609
TRAINING FINISHED
Epoch 1000/1000, Loss: 5.821947
MAPE: 0.094204
TRAINING FINISHED
Epoch 1000/1000, Loss: 6.225460
MAPE: 0.088274
TRAINING FINISHED
Epoch 1000/1000, Loss: 6.605380
MAPE: 0.100362
TRAINING FINIS

In [36]:
# Plotting results
fig = go.Figure()
fig.add_trace(
    go.Scatter(x=all_sizes, y=all_mapes,
               name='Все наблюдения', mode='markers')
)
fig.add_trace(
    go.Scatter(x=window_sizes, y=mapes_mean,
               name='Средние значения', mode='lines+markers')
)
fig.update_layout(
    title='Зависимость средней абсолютной ошибки от<br>размера скользящего окна',
    font=dict(size=20),
    xaxis=dict(title='Размер скользящего окна'),
    yaxis=dict(title='Средняя абсолютная ошибка'),
    width=1280,
    height=720
)
fig.show()

In [40]:
seq_length = 50
window_size = 5
batch_size = 1
hidden_size = 10
output_size = 3

sequence = list(arithmetic_progression(seq_length, 1, 1))
X, y = create_sliding_window_batches(sequence, window_size, batch_size, output_size)
X_train, y_train = X[0:int(len(X) * 0.8)], y[0:int(len(X) * 0.8)]
X_test, y_test = X[int(len(X) * 0.8):], y[int(len(X) * 0.8):][0:window_size]

verbosity = 5000
model = GRUModel(window_size, hidden_size, output_size)

# Параметры обучения
lr = 1e-2
epochs = 5000

model.train(x=X_train, y=y_train, lr=lr, max_epochs=epochs, verbosity=verbosity)

# y_pred, _, _ = model.forward(X_train)
# print(X_train.shape == y_pred.shape)
# print("Предсказание после обучения:\n", y_pred)
# print("Истинное значение:\n", y_train)
# print(f"MAPE: {mape(y_train, y_pred) * 100:.4f}%")

print('TEST')
y_pred, _, _ = model.forward(X_test)
print("Предсказания на тестовой выборке:\n", y_pred[-1])
print("Истинное значение на тестовой выборке:\n", y_test[-1])
print(f"MAPE: {mape(y_test[-1], y_pred[-1]) * 100:.4f}%")  

Epoch 5000/5000, Loss: 0.559535
MAPE: 0.033087
TRAINING FINISHED
Epoch 5000/5000, Loss: 0.559535
MAPE: 0.033087
TEST
Предсказания на тестовой выборке:
 [[42.04275696 43.0435765  44.08151996]]
Истинное значение на тестовой выборке:
 [[44 45 46]]
MAPE: 4.3222%
